# Naive Bayes

In [1]:
versioninfo()

Julia Version 1.11.4
Commit 8561cc3d68d (2025-03-10 11:36 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 8 × Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, skylake)
Threads: 8 default, 0 interactive, 4 GC (on 8 virtual cores)
Environment:
  LD_LIBRARY_PATH = /home/mhamdi/torch/install/lib:/home/mhamdi/torch/install/lib:/home/mhamdi/torch/install/lib:
  DYLD_LIBRARY_PATH = /home/mhamdi/torch/install/lib:/home/mhamdi/torch/install/lib:/home/mhamdi/torch/install/lib:
  JULIA_NUM_THREADS = 8


In [2]:
using Pkg; pkg"activate .."

  Activating project at `~/Work/git-repos/AI-ML-DL/jlai/Codes/Julia/Part-2`


In [3]:
using MLJ

In [4]:
import Languages, TextAnalysis

In [5]:
reviews = [
    ["I am furious! This is unacceptable.", "Negative"],
    ["Could you assist me with this problem? I'd appreciate it.", "Positive"],
    ["My coworker ignored me again. I hate this place.", "Negative"],
    ["The manager offered support. That was kind of her.", "Positive"],
    ["Thanks for your patience. You’re very understanding.", "Positive"],
    ["This is a disaster! Everything is going wrong.", "Negative"],
    ["The team handled the project well. Good job!", "Positive"],
    ["I’m so annoyed. Nobody ever helps me.", "Negative"],
    ["She gave me great advice. I’m grateful.", "Positive"],
    ["Why does this always happen? I’m done with this.", "Negative"],
    ["Your guidance was excellent. Thank you!", "Positive"],
    ["He yelled at me for no reason. So unfair.", "Negative"],
    ["The meeting went smoothly. No issues at all.", "Neutral"],
    ["I’m really happy with the results. Well done!", "Positive"],
    ["This company is terrible. I regret joining.", "Negative"],
]

15-element Vector{Vector{String}}:
 ["I am furious! This is unacceptable.", "Negative"]
 ["Could you assist me with this problem? I'd appreciate it.", "Positive"]
 ["My coworker ignored me again. I hate this place.", "Negative"]
 ["The manager offered support. That was kind of her.", "Positive"]
 ["Thanks for your patience. You’re very understanding.", "Positive"]
 ["This is a disaster! Everything is going wrong.", "Negative"]
 ["The team handled the project well. Good job!", "Positive"]
 ["I’m so annoyed. Nobody ever helps me.", "Negative"]
 ["She gave me great advice. I’m grateful.", "Positive"]
 ["Why does this always happen? I’m done with this.", "Negative"]
 ["Your guidance was excellent. Thank you!", "Positive"]
 ["He yelled at me for no reason. So unfair.", "Negative"]
 ["The meeting went smoothly. No issues at all.", "Neutral"]
 ["I’m really happy with the results. Well done!", "Positive"]
 ["This company is terrible. I regret joining.", "Negative"]

In [6]:
train, test = partition(1:length(reviews), .8; shuffle=true)

([10, 6, 14, 9, 12, 5, 2, 13, 3, 4, 8, 15], [7, 1, 11])

In [7]:
sentiment = [review[2] for review in reviews]
first(sentiment, 5)

5-element Vector{String}:
 "Negative"
 "Positive"
 "Negative"
 "Positive"
 "Positive"

In [8]:
y = coerce(sentiment, OrderedFactor)
scitype(y) <: AbstractVector{<:OrderedFactor}

true

In [9]:
tokens = [TextAnalysis.tokenize(Languages.English(), review[1]) for review in reviews]

15-element Vector{Vector{String}}:
 ["I", "am", "furious", "!", "This", "is", "unacceptable", "."]
 ["Could", "you", "assist", "me", "with", "this", "problem", "?", "I", "'", "d", "appreciate", "it", "."]
 ["My", "coworker", "ignored", "me", "again.", "I", "hate", "this", "place", "."]
 ["The", "manager", "offered", "support.", "That", "was", "kind", "of", "her", "."]
 ["Thanks", "for", "your", "patience.", "You", "’", "re", "very", "understanding", "."]
 ["This", "is", "a", "disaster", "!", "Everything", "is", "going", "wrong", "."]
 ["The", "team", "handled", "the", "project", "well.", "Good", "job", "!"]
 ["I", "’", "m", "so", "annoyed.", "Nobody", "ever", "helps", "me", "."]
 ["She", "gave", "me", "great", "advice.", "I", "’", "m", "grateful", "."]
 ["Why", "does", "this", "always", "happen", "?", "I", "’", "m", "done", "with", "this", "."]
 ["Your", "guidance", "was", "excellent.", "Thank", "you", "!"]
 ["He", "yelled", "at", "me", "for", "no", "reason.", "So", "unfair", "."]
 ["T

In [10]:
CountTransformer = @load CountTransformer pkg=MLJText
ct = CountTransformer()

[ Info: For silent loading, specify `verbosity=0`. 


import MLJText ✔


CountTransformer(
  max_doc_freq = 1.0, 
  min_doc_freq = 0.0)

In [11]:
mach_ct = machine(ct, tokens) |> fit!

[ Info: Training machine(CountTransformer(max_doc_freq = 1.0, …), …).


trained Machine; caches model-specific representations of data
  model: CountTransformer(max_doc_freq = 1.0, …)
  args: 
    1:	Source @058 ⏎ AbstractVector{AbstractVector{Textual}}


Matrix of counts

In [12]:
X = MLJ.transform(mach_ct, tokens)

15×97 LinearAlgebra.Adjoint{Int64, SparseArrays.SparseMatrixCSC{Int64, Int64}} with 147 stored entries:
⎡⠑⡗⠀⠸⠀⠀⡠⠌⢀⠀⠠⠈⠐⠂⠔⠀⠀⡈⠀⠀⠀⠄⠌⠐⠠⠠⠆⠤⡠⢂⠀⠀⠠⠀⠎⡀⡄⠐⠐⣀⎤
⎢⠃⡇⠑⢰⠐⠄⠀⠊⡀⠈⠄⡀⠂⠀⠀⣁⠂⠠⠡⠔⡀⠐⠈⠀⠂⡆⠆⠀⠀⠂⠀⠀⠂⠒⡀⠀⠐⢀⠁⢰⎥
⎣⠅⡂⠀⢢⠂⠐⠁⢂⠄⠁⠐⠀⠀⢂⠀⠠⠈⠂⠀⠈⠠⠀⢀⠂⢀⠄⠒⠀⠀⠀⠔⡰⠀⢠⠀⠐⠁⠢⠘⠠⎦

In [13]:
first(X, 5)

5-element Vector{Int64}:
 1
 0
 0
 0
 0

In [14]:
models("naive")

8-element Vector{NamedTuple{(:name, :package_name, :is_supervised, :abstract_type, :constructor, :deep_properties, :docstring, :fit_data_scitype, :human_name, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :inverse_transform_scitype, :is_pure_julia, :is_wrapper, :iteration_parameter, :load_path, :package_license, :package_url, :package_uuid, :predict_scitype, :prediction_type, :reporting_operations, :reports_feature_importances, :supports_class_weights, :supports_online, :supports_training_losses, :supports_weights, :target_in_fit, :transform_scitype, :input_scitype, :target_scitype, :output_scitype)}}:
 (name = BernoulliNBClassifier, package_name = MLJScikitLearnInterface, ... )
 (name = ComplementNBClassifier, package_name = MLJScikitLearnInterface, ... )
 (name = GaussianNBClassifier, package_name = MLJScikitLearnInterface, ... )
 (name = GaussianNBClassifier, package_name = NaiveBayes, ... )
 (name = MultinomialNBClassifier, package_name = ML

In [15]:
NaiveBayes = @load MultinomialNBClassifier pkg=NaiveBayes

[ Info: For silent loading, specify `verbosity=0`. 


import MLJNaiveBayesInterface ✔


MLJNaiveBayesInterface.MultinomialNBClassifier

In [16]:
nb = NaiveBayes()

MultinomialNBClassifier(
  alpha = 1)

In [17]:
mach_nb = machine(nb, X, y)

untrained Machine; caches model-specific representations of data
  model: MultinomialNBClassifier(alpha = 1)
  args: 
    1:	Source @596 ⏎ AbstractMatrix{Count}
    2:	Source @749 ⏎ AbstractVector{OrderedFactor{3}}


In [18]:
fit!(mach_nb, rows=train)

[ Info: Training machine(MultinomialNBClassifier(alpha = 1), …).


trained Machine; caches model-specific representations of data
  model: MultinomialNBClassifier(alpha = 1)
  args: 
    1:	Source @596 ⏎ AbstractMatrix{Count}
    2:	Source @749 ⏎ AbstractVector{OrderedFactor{3}}


Probabilistic predictions

In [19]:
y_prob = predict(mach_nb, rows=test) ## distributions

3-element CategoricalDistributions.UnivariateFiniteVector{OrderedFactor{3}, String, UInt32, Float64}:
 UnivariateFinite{OrderedFactor{3}}(Negative=>0.0616, Neutral=>0.639, Positive=>0.299)
 UnivariateFinite{OrderedFactor{3}}(Negative=>0.933, Neutral=>0.0155, Positive=>0.0519)
 UnivariateFinite{OrderedFactor{3}}(Negative=>0.128, Neutral=>0.298, Positive=>0.574)

In [20]:
pdf.(y_prob, "Positive") ## probabilities for "Positive"

3-element Vector{Float64}:
 0.29920973445433097
 0.05188738380654243
 0.574088964273517

In [21]:
log_loss(y_prob, y[test])

0.6104346626511568

Point predictions

In [22]:
ŷ = predict_mode(mach_nb, rows=test)

3-element CategoricalArrays.CategoricalArray{String,1,UInt32}:
 "Neutral"
 "Negative"
 "Positive"

In [23]:
ŷ .== sentiment[test] ## or `predict_mode(mach_nb, rows=test)`

3-element BitVector:
 0
 1
 1